NH

[algospot의 NH 링크](https://algospot.com/judge/problem/read/NH)  
  
### 문제  
음지에서 살아가는 보안 초전문가 슈퍼코더 K 는 이번에 새로운 은행의 전산망을 해킹하려고 합니다. 그에게는 원래 은행의 전산망을 해킹하는 것은 손바닥 뒤집는 것보다 쉬운 일인데, 2011년에 추가된 침입 감지 시스템(Intrusion Detection System, IDS) 때문에 골치를 앓고 있습니다. 이 침입 감지 시스템은 네트워크의 모든 트래픽을 감시하면서 그 중 특정 문자열을 발견하는 경우 곧장 해킹 시도가 있었음을 관리자에게 알려줍니다. 따라서 서버로 보내는 데이터에는 이 문자열이 포함되어 있으면 안 됩니다. 다행히 이 침입 감지 시스템이 인식하는 패턴들을 모두 이미 확보했습니다.  
  
이 전산망의 루트 비밀번호는 알파벳 소문자로 구성된 길이 n 의 문자열인데, 앞에서 이야기한 IDS 의 존재 때문에 해킹 시도로 인식되는 패턴들을 포함하는 비밀번호는 입력할 수 없습니다. 길이 n 인 소문자 문자열 중 IDS 에 인식되지 않는 문자열의 개수를 계산하는 프로그램을 작성하세요.  
  
### 출력  
IDS 에 인식되지 않는 문자열의 수를 출력합니다. 문자열의 수가 클 경우 10007 로 나눈 나머지를 출력합니다.  

solution의  
n : 루트 비밀번호의 길이 $(1 \leq n \leq 100)$  
patterns : IDS가 인식하는 패턴들을 저장한 리스트 $(1 \leq len(patterns) \leq 100)$  

각 패턴은 길이가 10 이하이다.  

In [1]:
# 아호-코라식 알고리즘(Aho-Corasick)을 적용한 trie 자료구조

# 아호-코라식 알고리즘은 KMP알고리즘의 확장판이라고 할 수 있으며,
# 이는 다중 문자열 검색 문제에 사용될 수 있다.
# ex) 'CACACHEFCACHY'에서 'CHCHE', 'HE', 'CHEF' 찾기

from queue import Queue
from copy import deepcopy

nodeCounter = 0

class TrieNode:
    no = 0 # 노드의 번호
    next_node = [] # 현 위치에서 알파벳 추가시 이동하는 노드
    children = [] # 자손 노드
    terminal = -1 # 현 위치에서 끝나는 문자열의 번호
    fail = None # 노드에 대응되는 문자열의 접미사이면서 트라이에 포함된 최대 문자열 노드
    output = [] # 이 노드가 방문되었을 때 나올 수 있는 탐색중인 부문 문자열들의 번호
    
    def __init__(self): # 트라이 노드 초기화
        self.no = -1
        self.next_node = [None for _ in range(26)] # A-Z를 각각 추가할 경우 이동할 노드
        self.children = [None for _ in range(26)] # A-Z까지에 대한 자손
        self.terminal = -1
        
    def __repr__(self):
        return "{"+"no : {}".format(self.no)+"}"
    
    def toNumber(self, word): # 현재 단어에 대해 확장할 자손 idx 반환
        apb = word[0]
        if apb.isalpha():
            answer = apb.capitalize()
            return ord(answer) - ord('A')
        else:
            return -1
    
    def insert(self, key='', string_id=0): # 자손 노드 추가
        if key == '': # 말단 노드
            self.terminal = string_id
        else: # 말단 노드가 아닐 경우
            next_node = self.toNumber(key)
            if self.children[next_node] == None:
                self.children[next_node] = TrieNode()
                # 새 노드 추가
            self.children[next_node].insert(key[1:], string_id)
            # 새 노드에 대해서 다시 재귀로 노드 추가
            
    def find(self, key=''): # 해당 문자열에 해당하는 노드 찾기
        if key == '':
            return self
        next_node = toNumber(key)
        if self.children[next_node] == None: # 해당하는 노드가 없다면 None 반환
            return None
        return self.children[next_node].find(key[1:]) # 재귀로 해당 노드 탐색
    
def computeFailFunc(root): # 실패 시에 돌아가는 노드에 대해서 계산
    q = Queue() # 너비 우선 탐색을 위한 큐
    root.fail = root # 문자열 매칭 실패시 root로 돌아가도록 초기화
    q.put(root) # 루트를 큐에 넣는다.
    while not q.empty(): # 너비 우선 탐색
        here = q.get() # 현재 탐색중인 노드
        for edge in range(26): # 모든 자식 노드에 대해서
            child = here.children[edge]
            if child is None: # 만약 해당 알파벳에 대한 자식 노드가 없다면 continue
                continue
            if here == root: # 만약 현재 탐색노드가 루트라면 자식노드가 실패시 루트노드로 돌아온다.
                child.fail = root
            else: # 루트노드가 아니라면
                t = here.fail
                while t != root and t.children[edge] == None:
                # 실패시 (돌아가는) 노드가 루트가 아니고, 실패 노드의 해당 자식노드가 없을 경우
                    t = t.fail
                    # 현재 노드의 실패 노드는 실패시 가는 노드가 실패했을 경우 돌아가는 노드와 동일
                if t.children[edge]:
                # 실패시 (돌아가는) 노드에 edge번 알파벳을 더한 노드가 존재할 경우
                    t = t.children[edge]
                    # 현재 노드가 실패시 돌아가는 노드는 실패시 노드 + edge번 알파벳을 더한 노드가 된다.
                child.fail = t
                # 자손의 실패 노드에 대해서는 현재노드의 실패노드로 초기화한다.
            child.output = deepcopy(child.fail.output)
            # 자손이 만들 수 있는 탐색 중인 부분 문자열들들에 대해
            # 실패 시 가는 노드로 만들 수 있는 탐색 중인 부분 문자열을 넣는다.
            if child.terminal != -1: # 자손에서 끝나는 문자열이 존재할 때
                child.output.append(child.terminal)
                # 자손의 아웃풋에 추가해준다.
            q.put(child) # 자손노드를 큐에 넣어주면서 너비 우선 탐색 진행
            
def ahoCorasick(string, root): # trie에 포함된 패턴들을 문자열에서 찾아준다.
    # (마지막 글자, 패턴 번호)를 가지는 리스트 반환
    ret = []
    state = root
    for i in range(len(string)):
        char = root.toNumber(string[i])
        while state != root and state.children[char] == None:
        # 루트 노드가 아니고, 노드의 해당 알파벳에 대한 자손노드가 없을 때까지(=실패)
            state = state.fail # 실패 후 노드로 상태를 옮긴다.
        if state.children[char]: # 만약 현 상태의 해당 알파벳에 대한 노드가 있을 때
            state = state.children[char] # 해당 알파벳 자식 노드 탐색
        for j in range(len(state.output)): # 상태의 output에 대해서
            ret.append((i, state.output[j]))
            # 매칭되는 부분 문자열들을 답에 넣는다.
    return ret # 답 반환

def count(length, state, mem): # 해당 패턴을 포함하지 않는 length길이의 문자열 개수 반환
    if len(state.output) > 0: # 해당 패턴을 포함할 때 0 리턴
        return 0
    if length == 0: # 해당 길이만큼 검사했을 때 패턴이 포함되지 않을 경우 1 리턴
        return 1
    if mem[length][state.no] != -1: # 만약 이미 계산된 상태라면
        return mem[length][state.no] # 메모이재이션 -> 해당 답 반환
    mem[length][state.no] = 0
    for letter in range(26): # 모든 알파벳에 대해서
        mem[length][state.no] += count(length-1, state.next_node[letter], mem)
        # 해당 알파벳을 추가한 상태에 대해서 재귀적으로 counting 진행
        mem[length][state.no] %= 10007
        # 수가 커질 경우 10007을 나눈 나머지
    return mem[length][state.no]

def computeTransition(here): # 현재 노드 상태에서 새로운 문자 추가시 다음으로 갈 노드 계산
    global nodeCounter # 노드 번호
    here.no = nodeCounter
    nodeCounter += 1
    for char in range(26): # 모든 알파벳에 대해서
        next_node = here
        while next_node != next_node.fail and next_node.children[char] == None:
        # 다음 노드와 다음 노드 실패 시 가는 노드가 다르고
        # 다음 노드에 알파벳 추가시에 갈 수 있는 노드가 있을 동안
            next_node = next_node.fail # 실패한 노드로 갱신
        if next_node.children[char]: # 만약 해당 알파벳을 추가할 수 있을 때
            next_node = next_node.children[char] # 해당 알파벳 추가한 상태 노드로 갱신
        here.next_node[char] = next_node
        # 현재 노드에서 알파벳 추가해서 갈 수 있는 노드 갱신
        if here.children[char]: # 현재 노드의 자식들에 대해서 재귀적으로 next_node 초기화
            computeTransition(here.children[char])

def solution(n, patterns):
    global nodeCounter
    nodeCounter = 0
    mem = [[-1 for _ in range(1001)] for _ in range(101)] 
    # 메모이재이션 [글자 길이][노드의 상태]
    trie = TrieNode() # 트라이 객체
    for i in range(len(patterns)):
        trie.insert(patterns[i], i) # 트라이에 패턴을 추가시킴 
    computeFailFunc(trie) # 해당 트라이 노드마다 실패 상황 계산
    computeTransition(trie) # 해당 트라이 노드마다 다음으로 갈 수 있는 노드 계산
    return count(n, trie, mem) # 패턴을 포함하지 않는 문자열 개수 계산

In [2]:
n = 2
patterns = ['rm', 'dd']

In [3]:
solution(n, patterns)

674

In [4]:
n = 4 
patterns = ['a', 'b', 'c', 'd']

In [5]:
solution(n, patterns)

4095

In [6]:
n = 100 
patterns = ['aa','ba','ab','cd'] 

In [7]:
solution(n, patterns)

5293